# U.S. K-12 Education Data Visualization

## Introduction
This is my final project for DTSA-5304 Fundamentals of Data Visualization. The visualizations in this project were created from [U.S. K-12 educational data](https://www.kaggle.com/datasets/noriuk/us-educational-finances) collected over a 25-year period from all fifty states and the District of Columbia. I approached these visualizations with an interest in exploring the relationship between educational funding and K-12 academic performance in the U.S.

## Target research question in this project
What is the relationship between annual U.S. K-12 educational expenditure and students' average academic performance viewed for each state + D.C.?

## Datasets used in this project
+ The [first dataset](https://www.kaggle.com/datasets/noriuk/us-educational-finances?select=states.csv) I used contains records of federal, state, and local spending on K-12 education in each of the 50 states + D.C. from 1992 to 2017. This information is assembled from annual reports published by the [U.S. Census Bureau](https://www.census.gov/programs-surveys/school-finances/library/publications.html). 

+ The [second dataset](https://www.kaggle.com/datasets/noriuk/us-educational-finances?select=naep.csv) contains National Assessment of Educational Progress ([NAEP](https://www.census.gov/programs-surveys/school-finances/library/publications.html)) scores for the 50 states + D.C. from 1990 - 2017. Commonly known as "the nation's report card", the NAEP measures proficiency in math and reading at the 4th and 8th grade levels. 

## Tools used in this project
I used [pandas](https://pandas.pydata.org/docs/index.html) and [Altair](https://altair-viz.github.io/) in Python to generate the visualizations. Details of the process are given in the next section.

## Data visualization and discussion

The segments below contain code for rendering the visualizations.

### 1. Initial data prep

We'll begin by importing the pandas and Altair libraries into the Jupyter notebook Python environment. We'll also import the [vega_datasets](https://github.com/vega/vega-datasets) module, which gives us access to a [topoJSON](https://github.com/topojson/topojson) map of the U.S. For our first visualization, we'll be using this map. 

In [2]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import altair as alt
from vega_datasets import data


We'll load the first data we're visualizing (`states.csv`) into a pandas dataframe and add two columns: an `id` column for aligning data with the topoJSON map, and a `TOTAL_EXPENDITURE_PER_STUDENT` column holding figures calculated for each state + D.C. from 1993 to 2017 (we drop 1992 because there is no enrollment data from that year). We'll preview the first five rows of the `states` dataframe to get a sense of what we're working with. Note that all monetary values represent 1/1000 of the actual dollar amount (in other words, multiply by $1000 to get the correct amount).

In [3]:
states = pd.read_csv("states.csv")
states.drop(states[states['YEAR'] == 1992].index, inplace=True)
states.reset_index(drop=True, inplace=True)
states['id'] = list(data.population_engineers_hurricanes()['id'][0:51]) * 25
states['TOTAL_EXPENDITURE_PER_STUDENT'] = states['TOTAL_EXPENDITURE']/states['ENROLL']
states.head()

,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE,id,TOTAL_EXPENDITURE_PER_STUDENT
0,Alabama,1993,727716.0,2827391.0,331409.0,1729295.0,766687.0,2833433,1564558.0,794146.0,237222.0,204207.0,1,3.893597
1,Alaska,1993,121156.0,1191398.0,176150.0,775829.0,239419.0,1126398,494917.0,433788.0,36291.0,135791.0,2,9.297088
2,Arizona,1993,676297.0,3427976.0,318465.0,1415407.0,1694104.0,3623946,1578889.0,1000914.0,164083.0,680139.0,4,5.358513
3,Arkansas,1993,311432.0,1346909.0,128196.0,771079.0,447634.0,1376067,782791.0,386526.0,68617.0,97824.0,5,4.418515
4,California,1993,5129788.0,28043338.0,2151157.0,17064146.0,8828035.0,28110986,15281147.0,8914559.0,1608514.0,1944760.0,6,5.479951


For these visualizations, we'll only be using the `STATE`, `YEAR`, `ENROLL`, `TOTAL_EXPENDITURE`, `id`, and `TOTAL_EXPENDITURE_PER_STUDENT` columns, so we'll remove the other columns from the `states` dataframe.

In [4]:
states.drop(columns=['TOTAL_REVENUE', 'FEDERAL_REVENUE', 'STATE_REVENUE', 'LOCAL_REVENUE', 'INSTRUCTION_EXPENDITURE', 'SUPPORT_SERVICES_EXPENDITURE', 'OTHER_EXPENDITURE', 'CAPITAL_OUTLAY_EXPENDITURE'], inplace=True)

An Altair object called `states_geo` holds the topoJSON map of the U.S.

In [5]:
states_geo = alt.topo_feature(data.us_10m.url, 'states')

### 2. Visualization A

Now we'll build our first visualization. Altair is used to create an interactive chart + choropleth map of the U.S. with color gradient representing the amount of K-12 spending per student in a given year in each state (herein we'll include D.C. as a "state"). You'll notice there is a slider on the bottom left where you can adjust the year. Try dragging the slider and you will see the choropleth map and chart dynamically update to reflect that year's K-12 spending per student in the 50 states + D.C.

In [6]:
slider = alt.binding_range(min=1993, max=2017, step=1, name='YEAR')
year = alt.selection_single(name="year", fields=['YEAR'],
                                bind=slider, init={'YEAR': 1993})

base = alt.Chart(states)

choropleth = base.mark_geoshape().encode(
    color=alt.Color('TOTAL_EXPENDITURE_PER_STUDENT:Q', 
                    legend=alt.Legend(title='Expenditure per Student (x $1000)', 
                    legendX=2, 
                    legendY=416, 
                    direction='horizontal', 
                    orient='none'), 
                    scale=alt.Scale(domain=[0, 33], scheme=alt.SchemeParams(name='yellowgreenblue', extent=[-.1, 1.3])))
).add_selection(year
).transform_filter(year
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(states_geo, key='id', fields=["type", "properties", "geometry"])
).project(
    type='albersUsa'
).properties(
    title='U.S. Educational Finances (K-12)',
    width=650,
    height=400)

year_text = base.mark_text(dy=-180, dx=-4, size=16).encode(
    text='YEAR:N'
).transform_filter(year)

label = base.mark_text(
    align="left",
    baseline="bottom",
    fontSize=14,
    fontWeight=600,
    color='black'
).encode(
    x=alt.value(2),
    y=alt.value(530),
    text=alt.value(["↓ drag the YEAR slider"]))

bar_chart = base.mark_bar().encode(
    x = alt.X('STATE', sort='-y'),
    y = alt.Y('TOTAL_EXPENDITURE_PER_STUDENT:Q', 
            scale=alt.Scale(domain=[0, 33]), 
            title='Expenditure per Student (x $1000)'),
    color='TOTAL_EXPENDITURE_PER_STUDENT:Q'
).transform_filter(year
).properties(
    width=600,
    height=400)

choropleth + year_text + label | bar_chart

alt.HConcatChart(...)

### 3. Secondary data prep

Next, we'll load the second dataset (`naep.csv`) into another pandas dataframe (`naep`). We'll remove rows containing NAEP scores from 1990 and 1992 because our `states` dataframe only has records going back to 1993. We'll also remove rows with NAEP scores for DoDEA ([Department of Defense Education Activity](https://www.dodea.edu/index.cfm)) since there is no corresponding enrollment or expenditure data in the `states` dataframe. Our goal is to merge the `naep` and `states` dataframes together. Looking at the first five rows of the `naep` dataframe, we'll need to do some reshaping to merge these data with the `states` row and column structure.

In [7]:
naep = pd.read_csv("naep.csv")
naep.drop(naep[naep['YEAR'] == 1992].index, inplace=True)
naep.drop(naep[naep['YEAR'] == 1990].index, inplace=True)
naep.drop(naep[naep['STATE'] == 'DoDEA'].index, inplace=True)
naep.head()

,YEAR,STATE,AVG_SCORE,TEST_SUBJECT,TEST_YEAR
0,2017,Alabama,232.170687741509,Mathematics,4
1,2017,Alaska,230.456277558902,Mathematics,4
2,2017,Arizona,234.435788152091,Mathematics,4
3,2017,Arkansas,233.848143678937,Mathematics,4
4,2017,California,232.262940994562,Mathematics,4


We'll use [MultiIndex](https://pandas.pydata.org/docs/user_guide/advanced.html#hierarchical-indexing-multiindex) in pandas to rearrange the `naep` dataframe so that each row carries a state's 4th and 8th grade math and reading NAEP scores for a particular year (NaN is substituted when no scores exist).

In [8]:
naep_reindex = naep.set_index(['YEAR', 'TEST_SUBJECT', 'TEST_YEAR', 'STATE'])
naep_reindex = naep_reindex.unstack(['TEST_YEAR','TEST_SUBJECT'])
naep_reindex = naep_reindex.reset_index()
naep_reindex.columns = naep_reindex.columns.to_flat_index()
naep_reindex.columns = ['YEAR', 'STATE', 'AVG_SCORE_4_Mathematics', 'AVG_SCORE_8_Mathematics', 'AVG_SCORE_4_Reading', 'AVG_SCORE_8_Reading']

We'll now merge the `naep_reindex` dataframe with the `states` dataframe. We'll also create a another merged dataframe with states sorted by K-12 expenditure per student.

In [9]:
states_naep_merged = pd.merge(states, naep_reindex, on=['STATE', 'YEAR'], how='outer')
states_naep_merged.replace(['—', '‡'], pd.NA, inplace=True)
states_naep_merged_sorted = states_naep_merged.sort_values(by=['YEAR', 'TOTAL_EXPENDITURE_PER_STUDENT'])

### 4. Visualization B

With the `states_naep_merged_sorted` dataframe, we will now use Altair to build another interactive visualization. Like the last visualization, this one has a time slider in the lower left. By dragging the slider, you can see the time progression of each state's expenditure per student along with the average NAEP scores for grade 4 mathematics updating over the 1993 - 2017 time period. As you adjust the time variable, you will notice the NAEP scores flash on and off at first. This is becuase the grade 4 NAEP math test was given only twice in the timeframe between 1993 and 2003. In 2003, the government started administering the NAEP every two years. 

Note that this visualization has been tweaked to make the NAEP score marks "persist" for one year beyond the year the test was issued. This is done to "smooth" the visual continuity between test scores. Be aware that the years for which the marks "persist" are actually without NAEP scores.

Also note how the histogram on the right shows the changing distribution of test scores. Visually, the trend for grade 4 mathematics NAEP scores is for the distribution to become narrower with an upward movement in the mean score over the time period from 1996 to 2017.

In [10]:
slider = alt.binding_range(min=1993, max=2017, step=1, name='YEAR')
year = alt.selection_single(name="year", fields=['YEAR'],
                                bind=slider, init={'YEAR': 1993})

base = alt.Chart(states_naep_merged_sorted)

bar_chart = base.mark_bar().encode(
    x = alt.X('STATE', axis=alt.Axis(orient='bottom'), sort=None),
    y = alt.Y('TOTAL_EXPENDITURE_PER_STUDENT:Q', 
            scale=alt.Scale(domain=[0, 33]), 
            title='Expenditure per Student (x $1000)'),
    color = alt.Color('TOTAL_EXPENDITURE_PER_STUDENT:Q',
                    legend=alt.Legend(title='Expenditure per Student (x $1000)',
                    legendX=2, 
                    legendY=400, 
                    direction='horizontal', 
                    orient='none'),
                    scale=alt.Scale(domain=[0, 33], scheme=alt.SchemeParams(name='yellowgreenblue', extent=[-.1, 1.3])))
).add_selection(year
).transform_filter(year
).properties(
    title='U.S. Educational Finances (K-12) + NAEP Scores Grade 4 Mathematics',
    width=600)

point_chart = base.transform_impute(
    impute='AVG_SCORE_4_Mathematics',
    key='YEAR',
    method='min',
    frame=[-1, 0],
    groupby=['STATE']
).mark_circle(color='red', size=35
).encode(
    x = alt.X('STATE', axis=alt.Axis(orient='bottom'), sort=None),
    y = alt.Y('AVG_SCORE_4_Mathematics:Q',
          axis=alt.Axis(title='● = Average NAEP Score Grade 4 Mathematics', titleColor='red', titleFontSize=12),
          scale=alt.Scale(domain=[170, 270]))
).transform_filter(year)

histogram = base.transform_impute(
    impute='AVG_SCORE_4_Mathematics',
    key='YEAR',
    method='min',
    frame=[-1, 0],
    groupby=['STATE']).mark_bar(color='red'
).encode(
    x = alt.X("AVG_SCORE_4_Mathematics:Q", 
            bin=alt.Bin(extent=[170, 270], step=2), 
            scale=alt.Scale(domain=[170, 270]), 
            axis=alt.Axis(title='NAEP Scores Grade 4 Mathematics (binned)', 
            titleColor='red', 
            titleFontSize=12)),
    y = alt.Y('count()', 
            scale=alt.Scale(domain=[0, 14]),
            axis=alt.Axis(title='# of States'))
).add_selection(year
).transform_filter(year
).properties(
    title='NAEP Grade 4 Mathematics Test Score Distribution')

(bar_chart + point_chart).resolve_scale(y='independent') | histogram


alt.HConcatChart(...)

### 5. Visualization C

We'll now apply the same interactive setup to visualize the average NAEP grade 4 reading scores between 1994 and 2017. Note how in this case, the mean score doesn't move as much as the mean mathematics score, but the distribution does appear to become narrower over time. Also note that the reading score scale on the right y-axis should be considered in isolation from the mathematics scale despite the same range being used for both scales. Since the math and reading tests measure different things, this puts into question how we should interpret comparisons between math and reading score movement.

In [11]:
slider = alt.binding_range(min=1993, max=2017, step=1, name='YEAR')
year = alt.selection_single(name="year", fields=['YEAR'],
                                bind=slider, init={'YEAR': 1993})

base = alt.Chart(states_naep_merged_sorted)

bar_chart = base.mark_bar().encode(
    x = alt.X('STATE', axis=alt.Axis(orient='bottom'), sort=None),
    y = alt.Y('TOTAL_EXPENDITURE_PER_STUDENT:Q', 
            scale=alt.Scale(domain=[0, 33]), 
            title='Expenditure per Student (x $1000)'),
    color = alt.Color('TOTAL_EXPENDITURE_PER_STUDENT:Q',
                    legend=alt.Legend(title='Expenditure per Student (x $1000)',
                    legendX=2, 
                    legendY=400, 
                    direction='horizontal', 
                    orient='none'),
                    scale=alt.Scale(domain=[0, 33], scheme=alt.SchemeParams(name='yellowgreenblue', extent=[-.1, 1.3])))
).add_selection(year
).transform_filter(year
).properties(
    title='U.S. Educational Finances (K-12) + NAEP Scores Grade 4 Reading',
    width=600)

point_chart = base.transform_impute(
    impute='AVG_SCORE_4_Reading',
    key='YEAR',
    method='min',
    frame=[-1, 0],
    groupby=['STATE']
).mark_circle(color='darkorange', size=35
).encode(
    x = alt.X('STATE', axis=alt.Axis(orient='bottom'), sort=None),
    y = alt.Y('AVG_SCORE_4_Reading:Q',
          axis=alt.Axis(title='● = Average NAEP Score Grade 4 Reading', titleColor='darkorange', titleFontSize=12),
          scale=alt.Scale(domain=[170, 270]))
).transform_filter(year)

histogram = base.transform_impute(
    impute='AVG_SCORE_4_Reading',
    key='YEAR',
    method='min',
    frame=[-1, 0],
    groupby=['STATE']
).mark_bar(color='darkorange'
).encode(
    x = alt.X("AVG_SCORE_4_Reading:Q", 
            bin=alt.Bin(extent=[170, 270], step=2), 
            scale=alt.Scale(domain=[170, 270]), 
            axis=alt.Axis(title='NAEP Scores Grade 4 Reading (binned)', 
            titleColor='darkorange', 
            titleFontSize=12)),
    y = alt.Y('count()', 
            scale=alt.Scale(domain=[0, 14]),
            axis=alt.Axis(title='# of States'))
).add_selection(year
).transform_filter(year
).properties(
    title='NAEP Grade 4 Reading Test Score Distribution')

(bar_chart + point_chart).resolve_scale(y='independent') | histogram

alt.HConcatChart(...)

### 6. Visualization D

Moving on, we'll use the same setup to visualize average NAEP grade 8 mathematics scores from 1996 to 2017. Note how the upward trend in mean score and narrowing distribution are reminiscent of what we saw in the grade 4 mathematics NAEP score data. Also note how the right y-axis and histogram x-axis have changed to a higher minimum and maximum (230-330) than the 4th grade minimum and maximum (170-270). NAEP reading and math scores are calculated on summative [0-500 scales](https://nces.ed.gov/nationsreportcard/guides/scores_achv.aspx). The max and min on these chart axes are arbitrarily chosen to achieve a workable range in which to view the data points for the given age group. 

In [12]:
slider = alt.binding_range(min=1993, max=2017, step=1, name='YEAR')
year = alt.selection_single(name="year", fields=['YEAR'],
                                bind=slider, init={'YEAR': 1993})

base = alt.Chart(states_naep_merged_sorted)

bar_chart = base.mark_bar().encode(
    x = alt.X('STATE', axis=alt.Axis(orient='bottom'), sort=None),
    y = alt.Y('TOTAL_EXPENDITURE_PER_STUDENT:Q', 
            scale=alt.Scale(domain=[0, 33]), 
            title='Expenditure per Student (x $1000)'),
    color = alt.Color('TOTAL_EXPENDITURE_PER_STUDENT:Q',
                    legend=alt.Legend(title='Expenditure per Student (x $1000)',
                    legendX=2, 
                    legendY=400, 
                    direction='horizontal', 
                    orient='none'),
                    scale=alt.Scale(domain=[0, 33], scheme=alt.SchemeParams(name='yellowgreenblue', extent=[-.1, 1.3])))
).add_selection(year
).transform_filter(year
).properties(
    title='U.S. Educational Finances (K-12) + NAEP Scores Grade 8 Mathematics',
    width=600)

point_chart = base.transform_impute(
    impute='AVG_SCORE_8_Mathematics',
    key='YEAR',
    method='min',
    frame=[-1, 0],
    groupby=['STATE']
).mark_circle(color='brown', size=35
).encode(
    x = alt.X('STATE', axis=alt.Axis(orient='bottom'), sort=None),
    y = alt.Y('AVG_SCORE_8_Mathematics:Q',
          axis=alt.Axis(title='● = Average NAEP Score Grade 8 Mathematics', titleColor='brown', titleFontSize=12),
          scale=alt.Scale(domain=[230, 330]))
).transform_filter(year)

histogram = base.transform_impute(
    impute='AVG_SCORE_8_Mathematics',
    key='YEAR',
    method='min',
    frame=[-1, 0],
    groupby=['STATE']
).mark_bar(color='brown'
).encode(
    x = alt.X('AVG_SCORE_8_Mathematics:Q', 
            bin=alt.Bin(extent=[230, 330], step=2), 
            scale=alt.Scale(domain=[230, 330]), 
            axis=alt.Axis(title='NAEP Scores Grade 8 Mathematics (binned)', 
            titleColor='brown', 
            titleFontSize=12)),
    y = alt.Y('count()', 
            scale=alt.Scale(domain=[0, 14]),
            axis=alt.Axis(title='# of States'))
).add_selection(year
).transform_filter(year
).properties(
    title='NAEP Grade 8 Mathematics Test Score Distribution')

(bar_chart + point_chart).resolve_scale(y='independent') | histogram

alt.HConcatChart(...)

### 7. Visualization E

Here we'll visualize the grade 8 NAEP reading score data from 1998 to 2017. The small upward movement of the mean is reminiscent of the 4th grade NAEP reading data visualization we saw earlier, although the distribution of the grade 8 NAEP reading scores appears to be relatively static.

In [13]:
slider = alt.binding_range(min=1993, max=2017, step=1, name='YEAR')
year = alt.selection_single(name="year", fields=['YEAR'],
                                bind=slider, init={'YEAR': 1993})

base = alt.Chart(states_naep_merged_sorted)

bar_chart = base.mark_bar().encode(
    x = alt.X('STATE', axis=alt.Axis(orient='bottom'), sort=None),
    y = alt.Y('TOTAL_EXPENDITURE_PER_STUDENT:Q', 
            scale=alt.Scale(domain=[0, 33]), 
            title='Expenditure per Student (x $1000)'),
    color = alt.Color('TOTAL_EXPENDITURE_PER_STUDENT:Q',
                    legend=alt.Legend(title='Expenditure per Student (x $1000)',
                    legendX=2, 
                    legendY=400, 
                    direction='horizontal', 
                    orient='none'),
                    scale=alt.Scale(domain=[0, 33], scheme=alt.SchemeParams(name='yellowgreenblue', extent=[-.1, 1.3])))
).add_selection(year
).transform_filter(year
).properties(
    title='U.S. Educational Finances (K-12) + NAEP Scores Grade 8 Reading',
    width=600)

point_chart = base.transform_impute(
    impute='AVG_SCORE_8_Reading',
    key='YEAR',
    method='min',
    frame=[-1, 0],
    groupby=['STATE']
).mark_circle(color='magenta', size=35
).encode(
    x = alt.X('STATE', axis=alt.Axis(orient='bottom'), sort=None),
    y = alt.Y('AVG_SCORE_8_Reading:Q',
          axis=alt.Axis(title='● = Average NAEP Score Grade 8 Reading', titleColor='magenta', titleFontSize=12),
          scale=alt.Scale(domain=[230, 330]))
).transform_filter(year)

histogram = base.transform_impute(
    impute='AVG_SCORE_8_Reading',
    key='YEAR',
    method='min',
    frame=[-1, 0],
    groupby=['STATE']
).mark_bar(color='magenta'
).encode(
    x = alt.X('AVG_SCORE_8_Reading:Q', 
            bin=alt.Bin(extent=[230, 330], step=2), 
            scale=alt.Scale(domain=[230, 330]), 
            axis=alt.Axis(title='NAEP Scores Grade 8 Reading (binned)', 
            titleColor='magenta', 
            titleFontSize=12)),
    y = alt.Y('count()', 
            scale=alt.Scale(domain=[0, 14]),
            axis=alt.Axis(title='# of States'))
).add_selection(year
).transform_filter(year
).properties(
    title='NAEP Grade 8 Reading Test Score Distribution')

(bar_chart + point_chart).resolve_scale(y='independent') | histogram

alt.HConcatChart(...)

### 8. Visualization F

Now we'll shift focus and view an array of graphs that show the expenditure per student + 4th grade NAEP math and reading scores in each state over the period from 1993 to 2017. Qualitatively, it's difficult to make state-specific determinations about expenditure per student as a predictor of NAEP test performance from these visualizations. However, the general trend indicates an increase in 4th grade NAEP math scores, and this at least coincides with a general increase in K-12 educational spending across the states over the 25-year period. 4th grade NAEP reading scores show less growth in general (with some exceptions).

You'll notice that in many states, the expenditure per student takes a dip somewhere in the 2009 - 2015 range. This is due to states' [reduction in school funding in reaction to the 2008 recesion](https://www.cbpp.org/research/state-budget-and-tax/a-punishing-decade-for-school-funding).

In [14]:
base = alt.Chart(states_naep_merged)

bar_chart = base.mark_bar().encode(
    x = alt.X('YEAR:O', axis=alt.Axis(orient='bottom'), sort=None),
    y = alt.Y('TOTAL_EXPENDITURE_PER_STUDENT:Q', 
            scale=alt.Scale(domain=[0, 33]), 
            title='Expenditure per Student (x $1000)'),
    color = alt.Color('TOTAL_EXPENDITURE_PER_STUDENT:Q',
                    legend=alt.Legend(title='Expenditure per Student (x $1000)',
                    legendX=2, 
                    legendY=220, 
                    direction='horizontal', 
                    orient='none'),
                    scale=alt.Scale(domain=[0, 33], scheme=alt.SchemeParams(name='yellowgreenblue', extent=[-.1, 1.3])))
).properties(
    width=240,
    height=160)

Grade_4_Math_point_chart = base.mark_circle(color='red').encode(
    x = alt.X('YEAR:O', axis=None),
    y = alt.Y('AVG_SCORE_4_Mathematics:Q',
    axis=alt.Axis(title='Grade 4 Math (red), Grade 4 Reading (orange)', titleFontSize=10),
    scale=alt.Scale(domain=[170, 270]))
).properties(
   width=240,
   height=160)

Grade_4_Reading_point_chart = base.mark_circle(color='darkorange').encode(
    x = alt.X('YEAR:O', axis=None),
    y = alt.Y('AVG_SCORE_4_Reading:Q',
    scale=alt.Scale(domain=[170, 270]), 
    axis=None)
).properties(
   width=240,
   height=160)

alt.concat(*(
    alt.layer(bar_chart, Grade_4_Math_point_chart, Grade_4_Reading_point_chart, title=f"{val}").transform_filter(alt.datum.STATE == val).resolve_scale(y='independent',x='independent')
    for val in states_naep_merged.STATE.unique()))

alt.ConcatChart(...)

### 9. Visualization G

We'll use the same graph setup to view expenditure per student figures + grade 8 NAEP math and reading scores over the 25 year period. Making intra-state comparisons between the 4th and 8th grade graphs, it would be interesting to find out how many of the students who took the 4th grade NAEP tests were in the same state when they took the 8th grade NAEP 4 years later. This points out that if we compare 4th grade and 8th grade scores, we might want to consider treating 4th grade data points as having a 4-year lead on 8th grade data points.

In [15]:
base = alt.Chart(states_naep_merged)

bar_chart = base.mark_bar().encode(
    x = alt.X('YEAR:O', axis=alt.Axis(orient='bottom'), sort=None),
    y = alt.Y('TOTAL_EXPENDITURE_PER_STUDENT:Q', 
            scale=alt.Scale(domain=[0, 33]), 
            title='Expenditure per Student (x $1000)'),
    color = alt.Color('TOTAL_EXPENDITURE_PER_STUDENT:Q',
                    legend=alt.Legend(title='Expenditure per Student (x $1000)',
                    legendX=2, 
                    legendY=220, 
                    direction='horizontal', 
                    orient='none'),
                    scale=alt.Scale(domain=[0, 33], scheme=alt.SchemeParams(name='yellowgreenblue', extent=[-.1, 1.3])))
).properties(
    width=240,
    height=160)

Grade_8_Math_point_chart = base.mark_circle(color='brown').encode(
    x = alt.X('YEAR:O', axis=None),
    y = alt.Y('AVG_SCORE_8_Mathematics:Q',
    axis=alt.Axis(title='Grade 8 Math (brown), Grade 8 Reading (magenta)', titleFontSize=10),
    scale=alt.Scale(domain=[230, 330]))
).properties(
   width=240,
   height=160)

Grade_8_Reading_point_chart = base.mark_circle(color='magenta').encode(
    x = alt.X('YEAR:O', axis=None),
    y = alt.Y('AVG_SCORE_8_Reading:Q',
    scale=alt.Scale(domain=[230, 330]), 
    axis=None)
).properties(
   width=240,
   height=160)

alt.concat(*(
    alt.layer(bar_chart, Grade_8_Math_point_chart, Grade_8_Reading_point_chart, title=f"{val}").transform_filter(alt.datum.STATE == val).resolve_scale(y='independent',x='independent')
    for val in states_naep_merged.STATE.unique()))

alt.ConcatChart(...)

### 10. Visualization H

The last visualization we'll look at in this report shows each state's total K-12 education expenditure plotted against the state's total K-12 enrollment, displayed for each year in a [faceted](https://lost-stats.github.io/Presentation/Figures/faceted_graphs.html) graph. If you hover the mouse over the plotted points, the state name will appear along with that state's average 4th and 8th grade NAEP reading and math scores for that year (if they are available).

There is a black regression line going through the points for each year. This regression line represents that year's expected total K-12 education expenditure for a given number of students. We can think of this line as representing the average K-12 education spending for a state of certain K-12 population size. Note that if a state is above the line, then that state's expenditure per student is above average for that year. If a state is below the line, then that state's expenditure per student is below average for that year.

In [16]:
base = alt.Chart(states_naep_merged)

point_graph = base.mark_circle().encode(
    x = alt.X('ENROLL:Q', title='# of K-12 students enrolled in a state'),
    y = alt.Y('TOTAL_EXPENDITURE:Q', title='K-12 expenditure (x $1000) in a state'),
    tooltip=['STATE', 'AVG_SCORE_4_Mathematics', 'AVG_SCORE_8_Mathematics', 'AVG_SCORE_4_Reading', 'AVG_SCORE_8_Reading']
).properties(
   width=240,
   height=160)

graph = point_graph + point_graph.transform_regression('ENROLL', 'TOTAL_EXPENDITURE').mark_line(color='black')

(graph).facet('YEAR', columns=5)

alt.FacetChart(...)

## Conclusion

In this report, we have reviewed several visualizations of a [U.S. K-12 educational finances dataset](https://www.kaggle.com/datasets/noriuk/us-educational-finances) and associated [NAEP score dataset](https://www.kaggle.com/datasets/noriuk/us-educational-finances?select=states.csv).

We have observed through interactive visualizations that over the 25-year period between 1993 and 2017, K-12 educational spending generally increased, as did average NAEP grade 4 mathematics scores. There is also some evidence of average NAEP grade 8 mathematics score increase. Average NAEP grade 4 reading scores on the whole show less increase in the 1993-2017 time period, as do average NAEP grade 8 reading scores. 

Our original research question:  
*What is the relationship between annual U.S. K-12 educational expenditure and students' average academic performance viewed for each state + D.C.?*

From this visual analysis, the best we can say is that the relationship between expenditure per student and NAEP scores at the state level remains unclear. This is not to say that there is no relationship between educational funding and student academic performance! On the contrary, there are many indications, such as the special case of the District of Columbia, that a positive correlation exists between educational funding and academic performance. Using these observations as starting points, more research would be needed at the district level nationwide to establish an emperical cause/effect relationship.

Some questions for further inquiry:

+ What would the change in educational spending be when adjusted for inflation?
+ What time lag effects are there between fiscal spending and educational performance?
+ What is the relationship between per student expenditure and average academic performance at the district level?